<a href="https://colab.research.google.com/github/Buchiexplores/Big_Data_Training/blob/master/HIVE_HBASE/Python_Notebooks/spotify_data_extraction_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ='fda982ead35c41c2b8b3e2763e7a0304'
secret = '7de8b0a3d1ca447f8db9d509ce8e5f1d'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
year = []
y_start = 1920  #Year Start
y_end = 2021  #Year End

for y in range(y_start,y_end,1):
  for i in range(0,2000,50):
      name = 'Define artist name' #Popularity by artist name

      track_results = sp.search('year:'+ str(y), type='track',limit=50,offset=i)
      for i, t in enumerate(track_results['tracks']['items']):
          artist_name.append(t['artists'][0]['name'])
          track_name.append(t['name'])
          track_id.append(t['id'])
          popularity.append(t['popularity'])
          year.append(y)
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 566.4748341019999


In [ ]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 172275


In [ ]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity,'year':year})
print(df_tracks.shape)
df_tracks.head()

(172275, 5)


,artist_name,track_name,track_id,popularity,year
0,Mamie Smith,Keep A Song In Your Soul,0cS0A1fUEUd1EW3FcF8AEI,15,1920
1,Screamin' Jay Hawkins,I Put A Spell On You,0hbkKFIJm7Z05H8Zl9w30f,9,1920
2,Oscar Velazquez,True House Music - Xavier Santos & Carlos Gomi...,19Lc5SfJJ5O1oaxY0fpwfh,17,1920
3,Mamie Smith & Her Jazz Hounds,Crazy Blues - 78rpm Version,3HnrHGLE9u2MjHtdobfWl9,5,1920
4,696,After Life,3ajZwXiT7qpanVm5DcvcQF,4,1920


In [ ]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172275 entries, 0 to 172274
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   artist_name  172275 non-null  object
 1   track_name   172275 non-null  object
 2   track_id     172275 non-null  object
 3   popularity   172275 non-null  int64 
 4   year         172275 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 6.6+ MB


In [ ]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

11787

In [ ]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [ ]:
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [ ]:
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()

artist_name    0
track_name     0
track_id       0
popularity     0
year           0
dtype: int64

In [ ]:
df_tracks.shape

(157758, 5)

In [ ]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 2
Time to run this code (in seconds): 327.9879792900001


In [ ]:
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 157756


In [ ]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (157756, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.598,0.224,5,-12.628,0,0.0936,0.991000,0.000522,0.3790,0.6340,149.976,audio_features,0cS0A1fUEUd1EW3FcF8AEI,spotify:track:0cS0A1fUEUd1EW3FcF8AEI,https://api.spotify.com/v1/tracks/0cS0A1fUEUd1...,https://api.spotify.com/v1/audio-analysis/0cS0...,168333,4
1,0.852,0.517,5,-7.261,0,0.0534,0.643000,0.026400,0.0809,0.9500,86.889,audio_features,0hbkKFIJm7Z05H8Zl9w30f,spotify:track:0hbkKFIJm7Z05H8Zl9w30f,https://api.spotify.com/v1/tracks/0hbkKFIJm7Z0...,https://api.spotify.com/v1/audio-analysis/0hbk...,150200,4
2,0.730,0.798,2,-7.311,1,0.0425,0.000173,0.801000,0.1280,0.0422,127.997,audio_features,19Lc5SfJJ5O1oaxY0fpwfh,spotify:track:19Lc5SfJJ5O1oaxY0fpwfh,https://api.spotify.com/v1/tracks/19Lc5SfJJ5O1...,https://api.spotify.com/v1/audio-analysis/19Lc...,422087,4
3,0.424,0.245,5,-11.470,1,0.0397,0.996000,0.799000,0.2350,0.4770,103.870,audio_features,3HnrHGLE9u2MjHtdobfWl9,spotify:track:3HnrHGLE9u2MjHtdobfWl9,https://api.spotify.com/v1/tracks/3HnrHGLE9u2M...,https://api.spotify.com/v1/audio-analysis/3Hnr...,198627,4
4,0.653,0.291,1,-16.198,0,0.0648,0.887000,0.000236,0.1050,0.3700,119.869,audio_features,3ajZwXiT7qpanVm5DcvcQF,spotify:track:3ajZwXiT7qpanVm5DcvcQF,https://api.spotify.com/v1/tracks/3ajZwXiT7qpa...,https://api.spotify.com/v1/audio-analysis/3ajZ...,106214,4


In [ ]:
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157756 entries, 0 to 157755
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   danceability      157756 non-null  float64
 1   energy            157756 non-null  float64
 2   key               157756 non-null  int64  
 3   loudness          157756 non-null  float64
 4   mode              157756 non-null  int64  
 5   speechiness       157756 non-null  float64
 6   acousticness      157756 non-null  float64
 7   instrumentalness  157756 non-null  float64
 8   liveness          157756 non-null  float64
 9   valence           157756 non-null  float64
 10  tempo             157756 non-null  float64
 11  type              157756 non-null  object 
 12  id                157756 non-null  object 
 13  uri               157756 non-null  object 
 14  track_href        157756 non-null  object 
 15  analysis_url      157756 non-null  object 
 16  duration_ms       15

In [ ]:
#columns_to_drop = ['analysis_url','track_href','type','uri']
#df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(157756, 18)

In [ ]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (157756, 18)


,artist_name,track_name,track_id,popularity,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Mamie Smith,Keep A Song In Your Soul,0cS0A1fUEUd1EW3FcF8AEI,15,1920,0.598,0.224,5,-12.628,0,0.0936,0.991000,0.000522,0.3790,0.6340,149.976,audio_features,spotify:track:0cS0A1fUEUd1EW3FcF8AEI,https://api.spotify.com/v1/tracks/0cS0A1fUEUd1...,https://api.spotify.com/v1/audio-analysis/0cS0...,168333,4
1,Screamin' Jay Hawkins,I Put A Spell On You,0hbkKFIJm7Z05H8Zl9w30f,9,1920,0.852,0.517,5,-7.261,0,0.0534,0.643000,0.026400,0.0809,0.9500,86.889,audio_features,spotify:track:0hbkKFIJm7Z05H8Zl9w30f,https://api.spotify.com/v1/tracks/0hbkKFIJm7Z0...,https://api.spotify.com/v1/audio-analysis/0hbk...,150200,4
2,Oscar Velazquez,True House Music - Xavier Santos & Carlos Gomi...,19Lc5SfJJ5O1oaxY0fpwfh,17,1920,0.730,0.798,2,-7.311,1,0.0425,0.000173,0.801000,0.1280,0.0422,127.997,audio_features,spotify:track:19Lc5SfJJ5O1oaxY0fpwfh,https://api.spotify.com/v1/tracks/19Lc5SfJJ5O1...,https://api.spotify.com/v1/audio-analysis/19Lc...,422087,4
3,Mamie Smith & Her Jazz Hounds,Crazy Blues - 78rpm Version,3HnrHGLE9u2MjHtdobfWl9,5,1920,0.424,0.245,5,-11.470,1,0.0397,0.996000,0.799000,0.2350,0.4770,103.870,audio_features,spotify:track:3HnrHGLE9u2MjHtdobfWl9,https://api.spotify.com/v1/tracks/3HnrHGLE9u2M...,https://api.spotify.com/v1/audio-analysis/3Hnr...,198627,4
4,696,After Life,3ajZwXiT7qpanVm5DcvcQF,4,1920,0.653,0.291,1,-16.198,0,0.0648,0.887000,0.000236,0.1050,0.3700,119.869,audio_features,spotify:track:3ajZwXiT7qpanVm5DcvcQF,https://api.spotify.com/v1/tracks/3ajZwXiT7qpa...,https://api.spotify.com/v1/audio-analysis/3ajZ...,106214,4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157756 entries, 0 to 157755
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   artist_name       157756 non-null  object 
 1   track_name        157756 non-null  object 
 2   track_id          157756 non-null  object 
 3   popularity        157756 non-null  int64  
 4   year              157756 non-null  int64  
 5   danceability      157756 non-null  float64
 6   energy            157756 non-null  float64
 7   key               157756 non-null  int64  
 8   loudness          157756 non-null  float64
 9   mode              157756 non-null  int64  
 10  speechiness       157756 non-null  float64
 11  acousticness      157756 non-null  float64
 12  instrumentalness  157756 non-null  float64
 13  liveness          157756 non-null  float64
 14  valence           157756 non-null  float64
 15  tempo             157756 non-null  float64
 16  type              15

In [ ]:
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,artist_name,track_name,track_id,popularity,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature


In [ ]:
df.to_csv('Spotify.csv')